# Imports

In [113]:
import pandas as pd
import requests
import numpy as np
import os 
from dotenv import load_dotenv
from dune_client.client import DuneClient
import random

from sklearn.linear_model import LogisticRegression

In [114]:
load_dotenv()

True

In [115]:
pd.options.display.float_format = '{:,.5f}'.format

In [116]:
def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)

In [117]:
seed = 20
set_random_seed(20)

# Pulling Data

In [118]:
api_key_dune = os.getenv("dune_key")
dune = DuneClient(api_key_dune)

In [119]:
def fetch_dune_data(num):
    result = dune.get_latest_result_dataframe(num)
    return result

top_voters_df = fetch_dune_data(4014335)
top_voters_df.to_csv('top_voters.csv', index=False)


In [120]:
top_voters_df = pd.read_csv('top_voters.csv')
top_voters_df.drop(columns=['first_vote','latest_vote'], inplace=True)

gen_voters_df = fetch_dune_data(4014573)
gen_voters_df.to_csv('gen_voters.csv', index=False)

proposal_df = fetch_dune_data(4023380)
proposal_df.to_csv('proposal_df.csv', index=False)

In [121]:
proposal_df = pd.read_csv('proposal_df.csv')
proposal_df.rename(columns={"proposer":"voter","proposals":"num_proposals"}, inplace=True)
proposal_df.head()

,voter,num_proposals
0,0xdcf7be2ff93e1a7671724598b1526f3a33b1ec25,42
1,0xb03e094e643a50e145e804b35787c28292e6afe4,10
2,0xb49c54edfe1cfe5ac83e74b179474f6224d0763f,2
3,0x1b6f3e48c7d235293f59b14d4c0a1ccba55411b4,2
4,0x87feed6162cb7dfe6b62f64366742349bf4d1b05,24


vote_freq_df = fetch_dune_data(4026611)
vote_freq_df.to_csv('vote_freq.csv', index=False)

In [122]:
vote_freq_df = pd.read_csv('vote_freq.csv')
vote_freq_df

,voter,avg_normalized_frequency
0,0xa3286502910574d30a4ef7d8d2a8609febc5e81d,0.03279
1,0x4d999f16ec6fd46a84e3c2cd4a9a64dd314ae829,0.01644
2,0xafa8d6fc711a792591a587e1908595747aad5895,0.01643
3,0x4c8d122bf527c47a74831c95d8ea06dd097b4ed9,0.01640
4,0xce389023479e7c3966ab1fcb3a779800d9f6b152,0.01639
...,...,...
87602,0xb27c628b8a5b93fe86e8efa0871bb5d3830de4b6,0.00001
87603,0xe1b0b110300080f147d55c30a139ad093e068407,0.00001
87604,0x7e39a589b7eb31d00a050c98370272a1a897bc7b,0.00001
87605,0xe86daa08bbdb721029cd20cf925552bc1bb10521,0.00001


In [123]:
gen_voters_df = pd.read_csv('gen_voters.csv')
gen_voters_df.describe()

,votes,avg_weight,num_delegated,percentage_delegated,num_dao_votes,total_gitcoin_donations,dex_trades_l60d,nft_mints_l60d,nfts_sold_l60d,nfts_bought_l60d
count,"78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000"
mean,3.73509,282.67497,236.88229,0.00000,0.35606,1.18598,1.53307,13.24502,0.13067,0.20039
std,3.21540,"37,119.80602","38,147.00352",0.00041,2.47424,13.79501,20.25268,777.12724,3.57946,4.19438
min,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
50%,2.00000,1.59000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
75%,6.00000,5.71000,1.17439,0.00000,0.00000,0.00000,1.00000,4.00000,0.00000,0.00000
max,13.00000,"10,118,377.26000","10,536,043.56961",0.11434,191.00000,"2,413.46620","5,275.00000","208,729.00000",450.00000,598.00000


# Data Aggregation & Feature Engineering

In [124]:
gen_voters_df = gen_voters_df.merge(vote_freq_df, on='voter',how='left').fillna(0)

In [125]:
gen_voters_df = gen_voters_df.merge(proposal_df, on='voter',how='left').fillna(0)

In [126]:
gen_voters_df.describe()

,votes,avg_weight,num_delegated,percentage_delegated,num_dao_votes,total_gitcoin_donations,dex_trades_l60d,nft_mints_l60d,nfts_sold_l60d,nfts_bought_l60d,avg_normalized_frequency,num_proposals
count,"78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000","78,827.00000"
mean,3.73509,282.67497,236.88229,0.00000,0.35606,1.18598,1.53307,13.24502,0.13067,0.20039,0.00006,0.00231
std,3.21540,"37,119.80602","38,147.00352",0.00041,2.47424,13.79501,20.25268,777.12724,3.57946,4.19438,0.00023,0.26337
min,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000
25%,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00002,0.00000
50%,2.00000,1.59000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00002,0.00000
75%,6.00000,5.71000,1.17439,0.00000,0.00000,0.00000,1.00000,4.00000,0.00000,0.00000,0.00005,0.00000
max,13.00000,"10,118,377.26000","10,536,043.56961",0.11434,191.00000,"2,413.46620","5,275.00000","208,729.00000",450.00000,598.00000,0.03279,40.00000


In [127]:
top_voters_df.describe()

,votes,avg_weight,num_delegated,percentage_delegated,num_dao_votes,total_gitcoin_donations,dex_trades_l60d,nft_mints_l60d,nfts_sold_l60d,nfts_bought_l60d,num_proposals,voting_frequency_ratio
count,"8,761.00000","8,761.00000","8,761.00000","8,761.00000","8,761.00000","8,761.00000","8,761.00000","8,761.00000","8,761.00000","8,761.00000","8,761.00000","8,761.00000"
mean,20.29962,"5,004.62139","5,732.80891",0.00006,1.03356,2.65024,3.74352,25.06449,0.35236,0.53350,0.06038,0.00015
std,6.59888,"96,454.84405","109,736.13066",0.00119,8.98561,26.92558,8.05276,121.51394,12.55400,14.50614,1.60141,0.00018
min,13.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00003
25%,16.00000,2.07000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00006
50%,18.00000,7.61000,1.65240,0.00000,0.00000,0.00000,0.00000,2.00000,0.00000,0.00000,0.00000,0.00011
75%,23.00000,33.93000,18.37893,0.00000,0.00000,0.00000,4.00000,29.00000,0.00000,0.00000,0.00000,0.00017
max,53.00000,"4,487,569.59000","5,216,754.77703",0.05661,479.00000,"1,046.30215",152.00000,"9,873.00000","1,029.00000","1,150.00000",58.00000,0.00560


In [128]:
gen_voters_df['ENS'] = gen_voters_df['name'].apply(lambda x: 1 if isinstance(x, str) and x.endswith('.eth') else 0)
top_voters_df['ENS'] = top_voters_df['name'].apply(lambda x: 1 if isinstance(x, str) and x.endswith('.eth') else 0)

In [129]:
top_voters_cor = top_voters_df.corr(numeric_only=True)
print(top_voters_cor['voting_frequency_ratio'].sort_values(ascending=False))

voting_frequency_ratio    1.00000
votes                     0.20640
dex_trades_l60d           0.07565
ENS                       0.04953
num_delegated             0.02693
percentage_delegated      0.02693
avg_weight                0.02613
nft_mints_l60d            0.02517
nfts_bought_l60d          0.00949
nfts_sold_l60d            0.00850
num_proposals             0.00804
num_dao_votes             0.00630
total_gitcoin_donations   0.00362
Name: voting_frequency_ratio, dtype: float64


In [130]:
print(top_voters_cor['votes'].sort_values(ascending=False))

votes                      1.00000
voting_frequency_ratio     0.20640
dex_trades_l60d            0.18717
num_delegated              0.08069
percentage_delegated       0.08068
avg_weight                 0.07761
num_proposals              0.03008
total_gitcoin_donations    0.02390
nft_mints_l60d             0.01987
num_dao_votes              0.00986
nfts_sold_l60d            -0.01035
nfts_bought_l60d          -0.01151
ENS                       -0.05351
Name: votes, dtype: float64


In [131]:
print(top_voters_cor['num_delegated'].sort_values(ascending=False))

num_delegated              1.00000
percentage_delegated       1.00000
avg_weight                 0.94036
num_proposals              0.35807
num_dao_votes              0.18538
total_gitcoin_donations    0.11842
votes                      0.08069
ENS                        0.04493
voting_frequency_ratio     0.02693
nfts_sold_l60d            -0.00140
nfts_bought_l60d          -0.00163
nft_mints_l60d            -0.00709
dex_trades_l60d           -0.02073
Name: num_delegated, dtype: float64


In [132]:
gen_voters_df

,voter,name,votes,avg_weight,num_delegated,percentage_delegated,num_dao_votes,total_gitcoin_donations,dex_trades_l60d,nft_mints_l60d,nfts_sold_l60d,nfts_bought_l60d,avg_normalized_frequency,num_proposals,ENS
0,0x8c1f91b04eeb01e781d0ab6f3154588b624fb866,<nil>,13,2.72000,2.72135,0.00000,0,0.00000,0,0,0,0,0.00009,0.00000,0
1,0xdca8ef8a999e6c6d7490b0ce7f86c645ba3aaf8e,<nil>,13,5.27000,5.65448,0.00000,0,0.00000,0,0,0,0,0.00003,0.00000,0
2,0xde6556218db4ce7ec3b24f37877c111d62095bd9,<nil>,13,20.09000,0.03692,0.00000,0,0.00000,12,29,0,0,0.00004,0.00000,0
3,0x2917634fd7319c38d377fff1d9ccac4505921339,haitangkeji.eth,13,0.23000,0.19259,0.00000,22,0.00000,0,0,0,0,0.00006,0.00000,1
4,0x7264636313534d00cf398ca682d0723336370fd0,<nil>,13,4.41000,4.65597,0.00000,0,0.00000,0,0,0,0,0.00013,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78822,0xfb2756ed668f60b75ab29c1d88ed3730a76dc866,loan.ensmaxisfam.eth,1,0.13000,0.00000,0.00000,0,0.00000,0,0,0,0,0.00002,0.00000,1
78823,0x4749b9793cc6539ca312aa92f699e04cde071f39,<nil>,1,2.94000,0.00000,0.00000,0,0.00000,0,0,0,0,0.00002,0.00000,0
78824,0x25b5293b472e5783f8cc5763e2b61009abf7e91a,hacker001.eth,1,4.09000,0.00000,0.00000,0,0.00000,0,0,0,0,0.00002,0.00000,1
78825,0xf91a49a9a931e0aa4535c6bf8a4162fef50db5e2,<nil>,1,3.11000,3.10519,0.00000,0,0.00000,0,0,0,0,0.00001,0.00000,0


In [133]:
gen_voters_cor = gen_voters_df.corr(numeric_only=True)
print(gen_voters_cor['avg_normalized_frequency'].sort_values(ascending=False))

avg_normalized_frequency    1.00000
votes                       0.07415
ENS                         0.03475
dex_trades_l60d             0.01502
nfts_sold_l60d              0.01102
nfts_bought_l60d            0.00998
num_proposals               0.00898
num_dao_votes               0.00725
avg_weight                  0.00671
num_delegated               0.00621
percentage_delegated        0.00620
nft_mints_l60d              0.00348
total_gitcoin_donations    -0.00026
Name: avg_normalized_frequency, dtype: float64


In [134]:
print(gen_voters_cor['votes'].sort_values(ascending=False))

votes                      1.00000
ENS                        0.07791
avg_normalized_frequency   0.07415
num_dao_votes              0.03940
nft_mints_l60d             0.01143
total_gitcoin_donations    0.00974
dex_trades_l60d            0.00933
avg_weight                 0.00873
nfts_sold_l60d             0.00871
num_delegated              0.00820
percentage_delegated       0.00818
num_proposals              0.00490
nfts_bought_l60d           0.00322
Name: votes, dtype: float64


In [135]:
print(gen_voters_cor['num_delegated'].sort_values(ascending=False))

num_delegated               1.00000
percentage_delegated        1.00000
avg_weight                  0.98970
num_proposals               0.02083
num_dao_votes               0.00835
votes                       0.00820
ENS                         0.00629
avg_normalized_frequency    0.00621
total_gitcoin_donations     0.00044
nft_mints_l60d             -0.00009
nfts_sold_l60d             -0.00020
nfts_bought_l60d           -0.00027
dex_trades_l60d            -0.00033
Name: num_delegated, dtype: float64


In [136]:
top_voters_df.columns

Index(['voter', 'name', 'votes', 'avg_weight', 'num_delegated',
       'percentage_delegated', 'num_dao_votes', 'total_gitcoin_donations',
       'dex_trades_l60d', 'nft_mints_l60d', 'nfts_sold_l60d',
       'nfts_bought_l60d', 'num_proposals', 'voting_frequency_ratio', 'ENS'],
      dtype='object')

In [137]:
gen_voters_df.rename(columns={"avg_normalized_frequency":"voting_frequency_ratio"}, inplace=True)

In [138]:
top_voters_df['type'] = 'top'
len(top_voters_df.columns)
# top_voters_df.columns

16

In [139]:
gen_voters_df['type'] = 'gen'
len(gen_voters_df.columns)
# gen_voters_df.columns

16

In [140]:
combined_df = pd.concat([top_voters_df, gen_voters_df], ignore_index=True)


In [141]:
combined_df.columns

Index(['voter', 'name', 'votes', 'avg_weight', 'num_delegated',
       'percentage_delegated', 'num_dao_votes', 'total_gitcoin_donations',
       'dex_trades_l60d', 'nft_mints_l60d', 'nfts_sold_l60d',
       'nfts_bought_l60d', 'num_proposals', 'voting_frequency_ratio', 'ENS',
       'type'],
      dtype='object')

# Correlation Analysis

In [142]:
combined_cor = combined_df.corr(numeric_only=True)
print(combined_cor['voting_frequency_ratio'].sort_values(ascending=False))

voting_frequency_ratio    1.00000
votes                     0.15118
ENS                       0.04624
dex_trades_l60d           0.02099
num_delegated             0.01306
percentage_delegated      0.01305
avg_weight                0.01293
num_dao_votes             0.01241
nfts_bought_l60d          0.01011
nfts_sold_l60d            0.01007
num_proposals             0.00942
nft_mints_l60d            0.00426
total_gitcoin_donations   0.00375
Name: voting_frequency_ratio, dtype: float64


In [143]:
print(combined_cor['votes'].sort_values(ascending=False))

votes                     1.00000
voting_frequency_ratio    0.15118
ENS                       0.10039
num_dao_votes             0.05908
num_delegated             0.04810
percentage_delegated      0.04807
avg_weight                0.04475
dex_trades_l60d           0.04028
num_proposals             0.03490
total_gitcoin_donations   0.03098
nfts_bought_l60d          0.01131
nfts_sold_l60d            0.01035
nft_mints_l60d            0.00982
Name: votes, dtype: float64


In [144]:
print(combined_cor['num_proposals'].sort_values(ascending=False))

num_proposals              1.00000
percentage_delegated       0.22970
num_delegated              0.22970
avg_weight                 0.12370
num_dao_votes              0.08755
votes                      0.03490
total_gitcoin_donations    0.03364
ENS                        0.01941
voting_frequency_ratio     0.00942
nft_mints_l60d            -0.00008
nfts_sold_l60d            -0.00042
nfts_bought_l60d          -0.00053
dex_trades_l60d           -0.00121
Name: num_proposals, dtype: float64


In [145]:
print(combined_cor['num_delegated'].sort_values(ascending=False))

num_delegated              1.00000
percentage_delegated       1.00000
avg_weight                 0.96610
num_proposals              0.22970
num_dao_votes              0.10437
votes                      0.04810
total_gitcoin_donations    0.04585
ENS                        0.01745
voting_frequency_ratio     0.01306
nft_mints_l60d            -0.00016
nfts_sold_l60d            -0.00041
nfts_bought_l60d          -0.00044
dex_trades_l60d           -0.00100
Name: num_delegated, dtype: float64


In [146]:
combined_df['voting_frequency_ratio'].describe()

count   87,588.00000
mean         0.00007
std          0.00023
min          0.00001
25%          0.00002
50%          0.00003
75%          0.00006
max          0.03279
Name: voting_frequency_ratio, dtype: float64

## Highly Engaged is Defined as Top 25% of Voters by Voting Frequency - This Will be the Target

In [147]:
engagement_threshold = combined_df['voting_frequency_ratio'].quantile(0.75)  # Top 25% threshold
combined_df['highly_engaged'] = combined_df['voting_frequency_ratio'].apply(lambda x: 1 if x > engagement_threshold else 0)


In [148]:
combined_df['highly_engaged']

0        1
1        1
2        1
3        1
4        1
        ..
87583    0
87584    0
87585    0
87586    0
87587    0
Name: highly_engaged, Length: 87588, dtype: int64

In [149]:
combined_df.columns

Index(['voter', 'name', 'votes', 'avg_weight', 'num_delegated',
       'percentage_delegated', 'num_dao_votes', 'total_gitcoin_donations',
       'dex_trades_l60d', 'nft_mints_l60d', 'nfts_sold_l60d',
       'nfts_bought_l60d', 'num_proposals', 'voting_frequency_ratio', 'ENS',
       'type', 'highly_engaged'],
      dtype='object')

In [150]:
numeric = combined_df.select_dtypes(include=['number'])

In [165]:
features

Index(['votes', 'avg_weight', 'num_delegated', 'percentage_delegated',
       'num_dao_votes', 'total_gitcoin_donations', 'dex_trades_l60d',
       'nft_mints_l60d', 'nfts_sold_l60d', 'nfts_bought_l60d', 'num_proposals',
       'ENS'],
      dtype='object')

In [166]:
target

'highly_engaged'

In [167]:
combined_df['voting_frequency_ratio'].describe()

count   87,588.00000
mean         0.00007
std          0.00023
min          0.00001
25%          0.00002
50%          0.00003
75%          0.00006
max          0.03279
Name: voting_frequency_ratio, dtype: float64

In [169]:
threshold = combined_df['voting_frequency_ratio'].quantile(0.75)  # Top 25%

# Feature selection
X = combined_df[features]  # Add more features as needed
y = (combined_df['voting_frequency_ratio'] > threshold).astype(int)  # Target variable

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Logistic Regression model
model.fit(X_train, y_train)

# Predict and add to dataframe
combined_df['highly_engaged'] = model.predict(X)

e:\Projects\steth_optimizer\steth_optimizer\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [170]:
combined_df[combined_df['highly_engaged']==1].describe()

,votes,avg_weight,num_delegated,percentage_delegated,num_dao_votes,total_gitcoin_donations,dex_trades_l60d,nft_mints_l60d,nfts_sold_l60d,nfts_bought_l60d,num_proposals,voting_frequency_ratio,ENS,highly_engaged
count,"27,575.00000","27,575.00000","27,575.00000","27,575.00000","27,575.00000","27,575.00000","27,575.00000","27,575.00000","27,575.00000","27,575.00000","27,575.00000","27,575.00000","27,575.00000","27,575.00000"
mean,12.38995,"2,341.77186","2,465.17558",0.00003,0.76464,1.53229,3.11267,25.73171,0.32286,0.48548,0.02528,0.00010,0.41617,1.00000
std,6.72114,"83,039.65266","89,384.86674",0.00097,6.11206,14.83371,34.07420,"1,259.34074",9.20447,10.64523,1.00563,0.00025,0.49293,0.00000
min,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,1.00000
25%,8.00000,1.59500,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00004,0.00000,1.00000
50%,10.00000,5.32000,0.12216,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00005,0.00000,1.00000
75%,15.00000,18.56000,4.49283,0.00000,0.00000,0.00000,2.00000,15.00000,0.00000,0.00000,0.00000,0.00011,1.00000,1.00000
max,53.00000,"10,118,377.26000","10,536,043.56961",0.11434,479.00000,"1,009.19125","5,275.00000","208,729.00000","1,029.00000","1,150.00000",58.00000,0.01644,1.00000,1.00000


In [172]:
from sklearn.metrics import classification_report, roc_auc_score

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the logistic regression model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Get probabilities for ROC AUC score

# Evaluate the model
print(classification_report(y_test, y_pred))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")


              precision    recall  f1-score   support

           0       0.85      0.78      0.81     13108
           1       0.47      0.58      0.52      4410

    accuracy                           0.73     17518
   macro avg       0.66      0.68      0.66     17518
weighted avg       0.75      0.73      0.74     17518

ROC AUC Score: 0.7653


e:\Projects\steth_optimizer\steth_optimizer\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [173]:
import numpy as np
import pandas as pd

# Get the feature names
feature_names = X_train.columns

# Get the coefficients from the model
coefficients = model.coef_[0]

# Create a DataFrame to hold the feature names and their corresponding coefficients
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Sort by absolute value of coefficients to see the most important features
feature_importance_df['Absolute Coefficient'] = np.abs(feature_importance_df['Coefficient'])
feature_importance_df = feature_importance_df.sort_values(by='Absolute Coefficient', ascending=False)

print(feature_importance_df)

                    Feature  Coefficient  Absolute Coefficient
0                     votes      0.19199               0.19199
11                      ENS     -0.16482               0.16482
10            num_proposals      0.07776               0.07776
4             num_dao_votes      0.01859               0.01859
6           dex_trades_l60d      0.01435               0.01435
9          nfts_bought_l60d      0.01090               0.01090
8            nfts_sold_l60d      0.00839               0.00839
5   total_gitcoin_donations     -0.00334               0.00334
7            nft_mints_l60d     -0.00002               0.00002
1                avg_weight      0.00001               0.00001
2             num_delegated      0.00000               0.00000
3      percentage_delegated     -0.00000               0.00000
